# This notebook attempts to extract all possible accounts from three *annual* financial reports: the balance sheet, income statement and cash flow statement.

### Why this work?
The output of this work will be used for a table mapping a Vietstock account to a hard-coded FAD account, which will be stored in our database.

### How is a Vietstock account defined?
We define a Vietstock account as a unique combination of its ID (denoted by the field `ReportNormID`), its English name (denoted by the field `NameEn`), and its Vietnamese name (denoted by the field `Name`).

In [29]:
import pandas as pd
import json
import glob
import re

### Here we define a function to simplify account names

In [30]:
def simplify_text(s):
    s1 = s.split(".")[-1]
    s1 = re.sub(r"\W+", " ", s1)
    s1 = " ".join(s1.split())
    s1 = s1.lower()
    return s1

### Define all necessary dictionaries

In [31]:
report_types = {"CDKT":{},
                "KQKD":{},
                "LC":{}}
lookup_dict_all = {"CDKT":{},
                    "KQKD":{},
                    "LC": {}}

### Process each report type sequentially
Before beginning the code, it's helpful to see what a financeInfo report file looks like:

```
[
    [
        {'year': 0000},
        {'year': 0001},
        ...
    ],
    {    
        'CashFlow Indirect': [
            {
                'reportnormID': 1234
            },
            {
                'reportnormID': 1234
            }
        ],
        'CashFlow Indirect': [
            {
                'reportnormID': 5678
            },
            {
                'reportnormID': 9000
            }
        ]
    },
    ...
]
```

In [32]:
for report_type, report_all_accounts in report_types.items():    
    for file in glob.glob(f'functions/schemaData/financeInfo/*{report_type}_Annual*.json'):
        with open(file, 'r') as jsonfile:
            try:
                j = json.load(jsonfile)
                for report_fullname, report_content in j[1].items():
                    if report_fullname not in lookup_dict_all[report_type].keys():
                        lookup_dict_all[report_type][report_fullname] = {}
                    for content in report_content:
                        acc_id = content['ReportNormID']
                        if acc_id not in lookup_dict_all[report_type][report_fullname].keys():
                            lookup_dict_all[report_type][report_fullname][acc_id] = content
            except Exception as e:
                print(f'EXCEPTION: {e}')
    
    for report_fullname, report_content in lookup_dict_all[report_type].items():
        if report_fullname not in report_all_accounts.keys():
            report_all_accounts[report_fullname] = []
        for acc_id, content in report_content.items():
            acc_n = simplify_text(content['NameEn'])
            acc_vi_n = simplify_text(content['Name'])
            parent_n = simplify_text(lookup_dict_all[report_type][report_fullname][content['ParentReportNormID']]['NameEn'])
            entry = (acc_n, parent_n, acc_vi_n)
            if entry not in report_all_accounts[report_fullname]:
                report_all_accounts[report_fullname].append(entry)
    
        with open(f'functions/schema/{report_type}_{report_fullname}_all_accounts.json', 'w') as writefile:
            json.dump(report_all_accounts, writefile, ensure_ascii=False, indent=4)
            
        d = pd.DataFrame(report_all_accounts[report_fullname])
        d.to_excel(f'functions/schema/{report_type}_{report_fullname}_all_accounts.xlsx')

### Quick view of our results

In [33]:
print(report_types['CDKT'])
print(report_types['KQKD'])

{'Balance Sheet': [('assets', 'assets', 'tài sản'), ('short term assets', 'total assets', 'tài sản ngắn hạn'), ('cash and cash equivalents', 'short term assets', 'tiền và các khoản tương đương tiền'), ('cash', 'cash and cash equivalents', 'tiền'), ('cash equivalents', 'cash and cash equivalents', 'các khoản tương đương tiền'), ('short term financial investments', 'short term assets', 'đầu tư tài chính ngắn hạn'), ('available for sale securities', 'short term financial investments', 'chứng khoán kinh doanh'), ('provision for diminution in value of available for sale securities', 'short term financial investments', 'dự phòng giảm giá chứng khoán kinh doanh'), ('held to maturity investments', 'short term financial investments', 'đầu tư nắm giữ đến ngày đáo hạn'), ('short term receivables', 'short term assets', 'các khoản phải thu ngắn hạn'), ('short term trade accounts receivable', 'short term receivables', 'phải thu ngắn hạn của khách hàng'), ('short term prepayments to suppliers', 'shor

In [34]:
report_types_expressdetails = {"CDKT":{},
                                "KQKD":{},
                                "LC":{}}
lookup_dict_all_expressdetails = {"CDKT":{},
                                    "KQKD":{},
                                    "LC": {}}

with open('functions/schema/bizType_ind_list.json', 'r') as jsonfile:
    biztypes_inds = sorted(json.load(jsonfile))
    jsonfile.close()

### balance sheet
for report_type, report_all_accounts in report_types_expressdetails.items():      
    for biztype_ind in biztypes_inds:
        biztype, ind = biztype_ind.split(";")[0], biztype_ind.split(";")[1]
        if biztype_ind not in lookup_dict_all_expressdetails[report_type].keys():
            lookup_dict_all_expressdetails[report_type][biztype_ind] = {}
        if biztype_ind not in report_types_expressdetails[report_type].keys():
            report_types_expressdetails[report_type][biztype_ind] = {}
        for file in glob.glob(f'functions/schemaData/financeInfo/{biztype}_{ind}*{report_type}_Annual*.json'):
            with open(file, 'r') as jsonfile:
                try:
                    j = json.load(jsonfile)
                    for report_fullname, report_content in j[1].items():
                        if report_fullname not in lookup_dict_all_expressdetails[report_type][biztype_ind].keys():
                            lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname] = {}
                        for content in report_content:
                            acc_id = content['ReportNormID']
                            if acc_id not in lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname].keys():
                                lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][acc_id] = content
                except Exception as e:
                    print(f'EXCEPTION: {e}')
                    
        for report_fullname, report_content in lookup_dict_all_expressdetails[report_type][biztype_ind].items():
            if report_fullname not in report_all_accounts[biztype_ind].keys():
                report_all_accounts[biztype_ind][report_fullname] = {}
            for acc_id, content in report_content.items():
                acc_n = simplify_text(content['NameEn'])
                acc_vi_n = simplify_text(content['Name'])
                parent_n = simplify_text(lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][content['ParentReportNormID']]['NameEn'])
                entry = f'{acc_n};{parent_n};{acc_vi_n}'

                if entry not in report_all_accounts[biztype_ind][report_fullname].keys():
                    report_all_accounts[biztype_ind][report_fullname][entry] = [acc_id]
                else:
                    report_all_accounts[biztype_ind][report_fullname][entry].append(acc_id)
                    
            for entry, ids in report_all_accounts[biztype_ind][report_fullname].items():
                if len(ids) > 1:
                    print("=== WARNING: multiple account IDs with similar entries ===")
                    print(f'{entry} ENTRY, {biztype_ind} biztype-ind, REPORT {report_type}-{report_fullname}')

            with open(f'functions/schema/{biztype_ind}_{report_type}_{report_fullname}_all_accounts.json', 'w') as writefile:
                json.dump(report_all_accounts[biztype_ind], writefile, ensure_ascii=False, indent=4)

=== WARNING: multiple account IDs with similar entries ===
;;nguyên giá ENTRY, Fund Management Company;Finance and Insurance biztype-ind, REPORT CDKT-Balance Sheet
=== WARNING: multiple account IDs with similar entries ===
;;giá trị hao mòn lũy kế ENTRY, Fund Management Company;Finance and Insurance biztype-ind, REPORT CDKT-Balance Sheet
=== WARNING: multiple account IDs with similar entries ===
;operating expenses; ENTRY, Security Company;Finance and Insurance biztype-ind, REPORT KQKD-Income Statement
=== WARNING: multiple account IDs with similar entries ===
;add non cash expenses; ENTRY, Security Company;Finance and Insurance biztype-ind, REPORT LC-CashFlow Indirect
=== WARNING: multiple account IDs with similar entries ===
cash at bank for securities company activities;add non cash expenses;tiền gửi ngân hàng cho hoạt động ctck ENTRY, Security Company;Finance and Insurance biztype-ind, REPORT LC-CashFlow Indirect
=== WARNING: multiple account IDs with similar entries ===
cash equiv